In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import Code.Nanoparticle as NP
import Code.FCCLattice as FCC
import Code.MutationOperator as MO
import numpy as np

from ase.visualize import view

In [3]:
Au_Descriptors = np.array([-13, -404, -301, -200]).transpose()
Ag_Descriptors = np.array([-1, -361, -289, -163]).transpose()
Cu_Descriptors = np.array([-26, 95, 147, 183]).transpose()

def runMonteCarlo(T, kozlovSymbol, descriptors):
    lattice = FCC.FCCLattice(21, 21, 21, 2)
    startParticle = NP.Nanoparticle(lattice)
    ratio = 0.5
    startParticle.kozlovSphere(6, ['Pd', kozlovSymbol], ratio)

    energies = list()
    bestParticles = list()

    oldEnergy = startParticle.getKozlovEnergy(descriptors, kozlovSymbol)

    energies.append(oldEnergy)
    bestParticles.append(startParticle)


    Pd_atoms = startParticle.getStoichiometry()['Pd']
    Other_atoms = startParticle.getStoichiometry()[kozlovSymbol]
    totalAtoms = len(startParticle.atoms)
    
    maxAttemptsToEscapeMinimum = 10*Pd_atoms*(totalAtoms - Pd_atoms)
    mutationOperator = MO.MutationOperator(min(Pd_atoms, Other_atoms))

    failedAttempts = 0
    
    acceptanceRate = 0
    beta = 8.6e-2*T
    while failedAttempts < maxAttemptsToEscapeMinimum:   
        failedAttempts = failedAttempts + 1
        
        newParticle = mutationOperator.mutateNanoparticle(startParticle)
        newEnergy = newParticle.getKozlovEnergy(descriptors, kozlovSymbol)

        deltaE = newEnergy - oldEnergy
        if deltaE < 0:
            acceptanceRate = 1
        else:
            acceptanceRate = np.exp(-beta*deltaE)

        if  np.random.random() > 1 - acceptanceRate:
            energies.append(newEnergy)
            bestParticles.append(newParticle)

            oldEnergy = newEnergy
            startParticle = newParticle
            
    return energies, bestParticles

In [4]:
def runMultipleMCSimulations(T, kozlovSymbol, descriptors, mcRuns):
    lowestEnergies = list()
    bestStructures = list()
    for mcRun in range(mcRuns):
        print("Run {0}".format(mcRun))
        energies, structures = runMonteCarlo(T, kozlovSymbol, descriptors)
        
        zippedResults = list(zip(energies, structures))
        zippedResults.sort(key=lambda x: x[0])
        energies, structures = zip(*zippedResults)
        
        lowestEnergies.append(energies[0])
        bestStructures.append(structures[0])
        
    zippedResults = list(zip(lowestEnergies, bestStructures))
    zippedResults.sort(key=lambda x: x[0])
    
    return lowestEnergies[0], bestStructures[0]

In [5]:
def visualizeMCRun(particle, kozlovSymbol):
    subsurfaceParticle = NP.Nanoparticle(particle.lattice)
    subsurfaceParticle.fromParticleData(particle.getAtoms(particle.getInnerAtomIndices()))

    coreParticle = NP.Nanoparticle(particle.lattice)
    coreParticle.fromParticleData(subsurfaceParticle.getAtoms(subsurfaceParticle.getInnerAtomIndices()))

    print("Descriptors for the Pd-{0} particle".format(kozlovSymbol))
    particle.printKozlovParameters(kozlovSymbol)
    print("_____________________________________________")
    
    print("Stoichiometry for the Pd-{0} subsurface particle".format(kozlovSymbol))
    print(subsurfaceParticle.getStoichiometry())
    print("_____________________________________________")
    
    print("Stoichiometry for the Pd-{0} core particle".format(kozlovSymbol))
    print(coreParticle.getStoichiometry())
    print("_____________________________________________")

    particleAtoms = particle.getASEAtoms()

    subsurfaceAtoms = subsurfaceParticle.getASEAtoms()
    subsurfaceAtoms.translate([20., 0, 0])

    coreAtoms = coreParticle.getASEAtoms()
    coreAtoms.translate([35, 0, 0])

    return coreAtoms + subsurfaceAtoms + particleAtoms

In [6]:
kozlovSymbol = 'Au'
lowestEnergy, particle = runMultipleMCSimulations(1, kozlovSymbol, Au_Descriptors, 5)

view(visualizeMCRun(particle, kozlovSymbol), viewer="x3d")


Run 0
Run 1
Run 2
Run 3
Run 4
Descriptors for the Pd-Au particle
number of heteroatomic bonds: 260.0
number of Au corner atoms: 24.0
number of Au edge atoms: 24.0
number of Au terrace atoms: 22.0
_____________________________________________
Stoichiometry for the Pd-Au subsurface particle
{'Pd': 44}
_____________________________________________
Stoichiometry for the Pd-Au core particle
{'Pd': 6}
_____________________________________________


In [7]:
kozlovSymbol = 'Ag'
lowestEnergy, particle = runMultipleMCSimulations(1, kozlovSymbol, Ag_Descriptors, 5)

view(visualizeMCRun(particle, kozlovSymbol), viewer="x3d")


Run 0
Run 1
Run 2
Run 3
Run 4
Descriptors for the Pd-Ag particle
number of heteroatomic bonds: 248.0
number of Ag corner atoms: 24.0
number of Ag edge atoms: 24.0
number of Ag terrace atoms: 22.0
_____________________________________________
Stoichiometry for the Pd-Ag subsurface particle
{'Pd': 44}
_____________________________________________
Stoichiometry for the Pd-Ag core particle
{'Pd': 6}
_____________________________________________


In [8]:
kozlovSymbol = 'Cu'
lowestEnergy, particle = runMultipleMCSimulations(1, kozlovSymbol, Cu_Descriptors, 5)

view(visualizeMCRun(particle, kozlovSymbol), viewer="x3d")


Run 0
Run 1
Run 2
Run 3
Run 4
Descriptors for the Pd-Cu particle
number of heteroatomic bonds: 355.0
number of Cu corner atoms: 12.0
number of Cu edge atoms: 17.0
number of Cu terrace atoms: 2.0
_____________________________________________
Stoichiometry for the Pd-Cu subsurface particle
{'Cu': 39, 'Pd': 5}
_____________________________________________
Stoichiometry for the Pd-Cu core particle
{'Pd': 4, 'Cu': 2}
_____________________________________________
